<a href="https://colab.research.google.com/github/B4Farouk/smpl-dr-clip/blob/main/simpledCLIP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import the GitHub Project

In [ ]:
!rm -rf project #remove if it already exists 
!git clone https://github.com/B4Farouk/smpl-dr-clip project

Cloning into 'project'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 125 (delta 66), reused 58 (delta 21), pack-reused 0
Receiving objects: 100% (125/125), 36.04 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (66/66), done.


### Install Dependencies
---
- pytorch (1.12.1)
- torchvision (0.13.1)
---
- pytorch3d
---
- SMPL
- chumpy

In [ ]:
import os
import sys

In [ ]:
# install torch
try:
    import torch
    if torch.__version__ != "1.12.1":
      raise ModuleNotFoundError()
except ModuleNotFoundError:
    !pip install torch==1.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# install torchvision
try:
    import torchvision
    if torchvision.__version__ != "0.13.1":
      raise ModuleNotFoundError()
except ModuleNotFoundError:
    !pip install torchvision==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# install pytorch3d
import torch, torchvision

try:
    import pytorch3d
except ModuleNotFoundError:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install pytorch3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.4 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221122-py3-none-any.whl size=61484 sha256=8a32f1196ecd0958b15677dff1690011777916cc2f4c5e8f88323962ac7f8004
  Stored in directory: /root/.cache/pip/wheels/2d/e4/d7/be0b4010933f5fffea6385e9b319eac9d6e56c82ee4a0164e5
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=ef8fce3c31b6195ed3eedbb60009b46c7317c18230a05dc1ccf9f0ec650526a6
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py37_cu113_pyt1121/download.html
     |████████████████████████████████| 47.2 MB 1.2 MB/s 


In [ ]:
# install SMPL dependencies
!pip install smplpytorch
!pip install chumpy

from smplpytorch.pytorch.smpl_layer import SMPL_Layer
from pytorch3d.structures import Meshes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 7.7 MB/s 
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58286 sha256=c16a0565847578e2b6606f16c0452778cd7d0df1b010eb93e3e71d7feeaf54c5
  Stored in directory: /root/.cache/pip/wheels/59/68/de/5e0c5d77e573e8c150e69e07a25035e6b6a04952d6e1814dbc
Successfully built chumpy


### GPU

In [ ]:
torch.cuda.is_available()

True

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

### SMPL

In [ ]:
# Create the SMPL layer
smpl_layer = SMPL_Layer(
model_root='/content/project')
#Beta is the (1,72) length pose argument and theta is the shape parameter (1,10) length argument
def SMPL(Beta,Theta):
   verts, Jtr=smpl_layer(Beta,Theta)
   return Meshes(verts,smpl_layer.th_faces[None, :])
# Generate T pose


torch.Size([1, 13776, 3])
torch.Size([1, 6890, 3])


In [ ]:
pose_params = torch.zeros(1, 72)
shape_params = torch.zeros(1, 10)
mesh = SMPL(pose_params,shape_params)


### Differentiable Renderer

In [ ]:
# Create the Differentiable Renderer layer
from project.rendering import CamerasFactory, Renderer

camsfactory = CamerasFactory(device=device)
cameras = camsfactory.fov_persp_scs(coords=(2, 30, 135), fov=60, frustrum_depth=(1, 100))
renderer = Renderer(device=device, cameras=cameras)

images = renderer.render(mesh)

TypeError: ignored

### CLIP

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
from project.clipModel import ClipModel

clip_model = ClipModel(model="ViT-B/32")
prompt = "a man standing up" # for example

similarity = clip.get_cosine_similarity(images, prompt)

print("The similarity between the 3d model and the prompt",prompt,"are",similarity)